In [1]:
import pandas as pd
import os
from collections import Counter

In [2]:
chat_dir = "ChzzkChat/chat/"

In [3]:
chat_logs = [os.path.join(chat_dir, chat_log) for chat_log in os.listdir(chat_dir)]

In [4]:
bad_lines = []
def bad_line_handler(line):
    bad_lines.append(line)
    return None

In [5]:
dfs = [
    pd.read_csv(chat_log, sep="\t", encoding="utf-8", header=None, on_bad_lines=bad_line_handler, engine='python')
    for chat_log in chat_logs if os.path.getsize(chat_log) != 0
]

In [6]:
bad_lines

[['탬탬버린', '2025-08-09 20:20:21', '채팅', '레인메iker', '대문자 ', '', 'E'],
 ['괴물쥐',
  '2025-08-07 00:42:52',
  '채팅',
  '하루살이 오빠 2705',
  '이 실력으로 어딜 간다고 ',
  '브론즈?'],
 ['파카', '2025-08-05 02:14:38', '채팅', '오이김치찜', '구으다가 라이즣', 'ㅏㄴ테 맞은건가'],
 ['풍월량', '2025-08-08 21:42:31', '채팅', '풍바사삭', '저장은 되어잇는거같은데 못불러오는 버그잇나', '보네'],
 ['울프', '2025-08-06 16:39:55', '채팅', '결대로', '울프님 ', '보시기엔 어때요?']]

In [7]:
chat_df = pd.concat(dfs, ignore_index=True)

In [8]:
chat_df.columns = ["streamer", "time", "type", "nickname", "chat"]

In [9]:
chat_df["chat"].values

array([nan, nan, nan, ..., '어디갓지...', '침착맨은 방송하라 방송하라 방송하라',
       '병거나 밥먹고 쉬자... 난 브레이크가 없어'], dtype=object)

In [10]:
chat_df.dropna(ignore_index=True, inplace=True)

In [11]:
import re

def simple_tokenizer(text):
    # 1. 소문자 변환 (원하면)
    text = text.lower()
    
    # 2. 이모티콘 / 반복 문자 패턴 처리
    text = re.sub(r"ㅋ{2,}", " <laugh> ", text)
    text = re.sub(r"[ㅠㅜ]{2,}", " <cry> ", text)
    
    # 3. 특수문자 → 공백
    text = re.sub(r"[^가-힣\s]", " ", text)
    
    # 4. 공백 기준 토큰 분리
    tokens = text.split()
    return tokens

# ✅ 테스트
print(simple_tokenizer("ㅋㅋㅋㅋ 이건 ㄹㅇ 대박 ㅠㅠ!! 헐ㅋㅋ"))

['이건', '대박', '헐']


#### 채팅, 후원 비율

In [12]:
chat_df["type"].value_counts(normalize=True) * 100

type
채팅    99.503998
후원     0.496002
Name: proportion, dtype: float64

#### 스트리머별 채팅 비율

In [13]:
chat_df["streamer"].value_counts(normalize=True) * 100

streamer
한동숙           10.439841
풍월량           10.202747
텐코 시부키         9.785309
울프             7.176495
시라유키 히나        6.620978
아야츠노 유니        6.423637
아라하시 타비        6.233266
괴물쥐            5.804524
하나코 나나         5.771093
탬탬버린           5.745428
아카네 리제         4.488384
강지             3.710071
파카             3.403749
침착맨            3.401634
네네코 마시로        2.868664
아오쿠모 린         2.836007
서새봄냥 SEBOM     2.814069
다주             2.274106
Name: proportion, dtype: float64

In [14]:
chat_df.head()

,streamer,time,type,nickname,chat
0,다주,2025-08-06 15:01:06,채팅,하이안D,다하다하
1,다주,2025-08-06 15:01:08,채팅,주다영진심녀,다하다하
2,다주,2025-08-06 15:01:16,채팅,주다영진심녀,반갑다주
3,다주,2025-08-06 15:01:57,채팅,요리하는개백수탈출함,젠지 pc방
4,다주,2025-08-06 15:02:11,채팅,하이안D,젠지 PC방 진짜 궁금해요 ~


In [15]:
chat_df["token"] = chat_df["chat"].apply(simple_tokenizer)

In [16]:
result = chat_df.groupby('streamer')['token'].agg(lambda x: [item for sublist in x for item in sublist]).reset_index()

In [17]:
with open('ChzzkChat/stopwords-ko.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

In [18]:
chat_df["streamer"].unique()

array(['다주', '서새봄냥 SEBOM', '탬탬버린', '아라하시 타비', '괴물쥐', '텐코 시부키', '강지',
       '한동숙', '시라유키 히나', '파카', '풍월량', '울프', '아오쿠모 린', '네네코 마시로', '아카네 리제',
       '아야츠노 유니', '하나코 나나', '침착맨'], dtype=object)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
documents = result['token'].apply(lambda x : ' '.join(x)).values
streamers = result["streamer"]

In [21]:
# TfidfVectorizer 사용하여 TF-IDF 계산, 불용어 제외
vectorizer = TfidfVectorizer(stop_words=stopwords)

# TF-IDF 행렬을 계산합니다
X = vectorizer.fit_transform(documents)

# TF-IDF 결과를 데이터프레임으로 변환
df_tfidf = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out(), index=streamers)

# 각 스트리머별로 상위 10개 단어 추출
top_10_words = {}

for streamer in df_tfidf.index:
    # 각 스트리머의 TF-IDF 값을 기준으로 상위 10개 단어 추출
    top_10 = df_tfidf.loc[streamer].sort_values(ascending=False).head(10)
    top_10_words[streamer] = top_10.index.tolist()

# 결과 출력
for streamer, words in top_10_words.items():
    print(f"{streamer}의 상위 10개 단어: {', '.join(words)}")

/home/kse/anaconda3/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['sebom', '가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '나나', '낫다', '네네코', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '리제', '마시로', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서새봄냥', '서술한바와같이', '시라유키', '시부키', '쓰여', '아니다', '아니라', '아라하시', '아야츠노', '아오쿠모', '아카네', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '유니', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '타비', '텐코', '틀림없다', '편이', '하고', '하기', '하기만', '하나코', '하는', '하는것만', '하는것이', '하다', '하면', '하

강지의 상위 10개 단어: 강하, 어어, 강지님, 이게, 강순이, 그건, 어라, 이거, 이제, 이야
괴물쥐의 상위 10개 단어: 물쥐야, 물쥐님, 카타, 갱플, 대물쥐, 카르마, 물쥐, 카선족, 아칼리, 지금
네네코 마시로의 상위 10개 단어: 안냐냐, 시로, 바냐냐, 오오, 너무, 오옹, 찌로, 시로가, 뭐야, 마시로
다주의 상위 10개 단어: 다하, 다바다바, 다주님, 옥슈슈, 너무, 네즈코, 아님, 기흥, 다하다하, 왓더버거
서새봄냥 SEBOM의 상위 10개 단어: 복돌이, 너무, 어우, 이거, 이게, 스승님, 새봄님, 마피아, 복도링, 역시
시라유키 히나의 상위 10개 단어: 히나, 빠히나, 히나야, 디아루가, 어어, 오오, 너무, 히나가, 그건, 네넹
아라하시 타비의 상위 10개 단어: 타비, 아라하시, 타비야, 어어, 어우, 약머거약, 채팅, 그건, 이게, 꼴림
아야츠노 유니의 상위 10개 단어: 유니, 아야츠노, 야꾸머거, 채팅, 유니야, 유니가, 유니우승, 일루와잇, 노력합시다, 안녕하시지
아오쿠모 린의 상위 10개 단어: 어어, 어우, 오우, 빠이린, 오오, 모린, 그건, 쿠모린, 우어터, 린이
아카네 리제의 상위 10개 단어: 리제, 콘츕츕, 네네, 리제야, 엥나, 리제는, 레샤, 리제가, 너무, 그러게
울프의 상위 10개 단어: 유나라, 너무, 빅버지, 든프, 이게, 라이즈, 빅토르, 이거, 어우, 이건
침착맨의 상위 10개 단어: 궤도, 단군, 이광수, 없으면, 침착맨, 광수, 비행기, 광수형, 너무, 상하이
탬탬버린의 상위 10개 단어: 그건, 채팅, 유나라, 너무, 이게, 탬바, 이거, 이건, 탬탬보리인, 이제
텐코 시부키의 상위 10개 단어: 부키야, 부키, 어어, 너무, 오오, 누넴, 마자, 그러게, 부키가, 어우
파카의 상위 10개 단어: 파카님, 갱플, 카타, 아칼리, 다팔고, 크산테, 유미, 구인수, 어어, 탄성팔
풍월량의 상위 10개 단어: 너무, 이거, 풍형, 풍바, 이게, 어어, 풍하, 마피아, 이제, 뭐야
하나코 나나의